In [2]:
import pandas as pd

# Version 1
Quick test -- output each country as its own files

In [3]:
q = '''SELECT
  date(_partitiontime) AS date,
  flag,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
GROUP BY
  date,
  flag
ORDER BY
  flag,
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

In [4]:
df.head()

,date,flag,fishing_hours
0,2015-08-17,MNG,24
1,2015-08-18,MNG,24
2,2015-08-19,MNG,23
3,2015-08-20,MNG,24
4,2015-08-21,MNG,22


In [5]:
len(df)

164185

In [6]:
df = df.sort_values(by=['flag', 'date']).reset_index(drop=True)
df = df.fillna(0)

In [7]:
flag = df.flag[0]
iso3s = []
header = "date,fishing\n"
rows = []
for index, row in df.iterrows():
    fl = row.flag
    if fl!=flag:
        with open("countries/{}.csv".format(flag), 'w') as f:
            f.write(header)
            f.write("\n".join(rows))
        rows = []
#         print flag
        iso3s.append(flag)
        flag = fl
    
    rows.append("{},{}".format(row.date, row.fishing_hours))
    
with open("countries/{}.csv".format(flag), 'w') as f:
    f.write(header)
    f.write("\n".join(rows))
rows = []
# print flag
iso3s.append(flag)


In [7]:
q = '''SELECT
  date(_partitiontime) AS date,
  flag,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
  where _partitiontime >= timestamp("2016-01-01")
GROUP BY
  date,
  flag
ORDER BY
  flag,
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

In [8]:
df = df.sort_values(by=['flag', 'date']).reset_index(drop=True).fillna(0)

In [10]:
flag = df.flag[0]
iso3s = []
header = "date,fishing\n"
rows = []
for index, row in df.iterrows():
    fl = row.flag
    if fl!=flag:
        with open("countries2016/{}.csv".format(flag), 'w') as f:
            f.write(header)
            f.write("\n".join(rows))
        rows = []
#         print flag
        iso3s.append(flag)
        flag = fl
    
    rows.append("{},{}".format(row.date, row.fishing_hours))
    
with open("countries/{}.csv".format(flag), 'w') as f:
    f.write(header)
    f.write("\n".join(rows))
rows = []
# print flag
iso3s.append(flag)

In [12]:
q = '''SELECT
  date(_partitiontime) AS date,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
GROUP BY
  date
ORDER BY
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

In [13]:
df = df.sort_values(by='date').reset_index(drop=True).fillna(0)

In [14]:
with open("countries/WLD.csv".format(flag), 'w') as f:
    f.write(header)
    rows = []
    for index, row in df.iterrows():
        rows.append("{},{}".format(row.date,row.fishing_hours))
    f.write("\n".join(rows))

In [ ]:
q = '''SELECT
  date(_partitiontime) AS date,
  integer(SUM(fishing_hours)) fishing_hours
FROM
  [world-fishing-827:gfw_draft_data.fishing_effort]
GROUP BY
  date
ORDER BY
  date'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False)

# Version 2 
 - put all the values in the same file
 - use local time for the days
 - include fishing vessel days

In [7]:
q = '''SELECT
  date,
  flag,
  COUNT(*) vessels_fishing,
  SUM(fishing_hours) fishing_hours
FROM (
  SELECT
    mmsi,
    DATE(SEC_TO_TIMESTAMP(TIMESTAMP_TO_SEC(timestamp) + lon/180*12*3600)) date,
    SUM(hours) fishing_hours,
  FROM
    [gfw_research.nn2]
  WHERE
    nnet_score2 = 1
  GROUP BY
    mmsi,
    date) a
LEFT JOIN
  [gfw_draft_data.fishing_vessels] b
ON
  a.mmsi = b.mmsi
  where flag not in ("CUB","MNG")
  and flag is not null
GROUP BY
  flag,
  date
ORDER BY
  flag,'''
df = pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose=False).set_index("date")

In [5]:
len(df)

137127

In [8]:
df.to_csv("allcountries2012-2016.csv")